# finetuning 

In [1]:
import json
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType
import os

MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
OUTPUT_DIR = "./phi3_medical_finetuned"
DATA_FILE = "/kaggle/input/hydrogen/data.json"

def load_and_prepare_data():
    """Charger et préparer les données"""
    print("Chargement des données...")
    
    with open(DATA_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    
    prompts = []
    for item in data:
        user_input = item['texte']
        classification = item['classification']
        entites = item['entites']
        
        response = {
            "classification": classification,
            "heure": entites.get('heure', 'NON_SPECIFIE'),
            "date": entites.get('date', 'NON_SPECIFIE'),
            "nom_docteur": entites.get('nom_docteur', 'NON_SPECIFIE')
        }
        
        prompt = f"""<|user|>
Analyse cette demande médicale et réponds en JSON:
{user_input}
<|end|>
<|assistant|>
{json.dumps(response, ensure_ascii=False)}
<|end|>"""
        
        prompts.append(prompt)
    
    split_idx = int(len(prompts) * 0.8)
    train_prompts = prompts[:split_idx]
    val_prompts = prompts[split_idx:]
    
    print(f"Train: {len(train_prompts)}, Validation: {len(val_prompts)}")
    return train_prompts, val_prompts

def setup_model():
    print("Chargement du modèle Phi-3...")
    
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32,  
        trust_remote_code=True,
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
        print(f"Modèle sur GPU: {torch.cuda.get_device_name()}")
    
    for param in model.parameters():
        param.requires_grad = False  
    
    # Configuration LoRA
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=16,  
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["qkv_proj", "o_proj"]  
    )
    
    # Appliquer LoRA
    model = get_peft_model(model, lora_config)
    
    for name, param in model.named_parameters():
        if 'lora_' in name:
            param.requires_grad = True
    
    model.print_trainable_parameters()
    
    return model, tokenizer

def create_datasets(train_prompts, val_prompts, tokenizer):
    """Créer les datasets tokenizés"""
    print("Création des datasets...")
    
    def tokenize_function(examples):
        """Fonction de tokenization"""
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors=None,
        )
        
        tokenized["labels"] = tokenized["input_ids"].copy()
        
        return tokenized
    
    train_dataset = Dataset.from_dict({"text": train_prompts})
    val_dataset = Dataset.from_dict({"text": val_prompts})
    
    print("Tokenization...")
    
    train_dataset = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
        desc="Tokenizing train"
    )
    
    val_dataset = val_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
        desc="Tokenizing validation"
    )
    
    print("Datasets tokenizés")
    return train_dataset, val_dataset

def train_model(model, tokenizer, train_dataset, val_dataset):
    print("Début de l'entraînement...")
    
    trainable_params = [name for name, param in model.named_parameters() if param.requires_grad]
    print(f"Paramètres entraînables: {len(trainable_params)}")
    
    if len(trainable_params) == 0:
        raise ValueError("Aucun paramètre entraînable trouvé!")
    
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=3,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,  
        warmup_steps=10,
        logging_steps=5,
        save_steps=50,
        save_total_limit=1,
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        dataloader_num_workers=0,
        report_to=[],
        fp16=False,  
        optim="adamw_torch",
        label_names=["labels"],
        gradient_checkpointing=False, 
    )
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        processing_class=tokenizer,
        data_collator=data_collator,
    )
    
    # Entraînement
    trainer.train()
    
    # Sauvegarde
    trainer.save_model()
    tokenizer.save_pretrained(OUTPUT_DIR)
    
    print(f"Modèle sauvegardé dans {OUTPUT_DIR}")



In [2]:
def main():
    print("FINE-TUNING PHI-3")
    print("=" * 50)
    
    # Nettoyage mémoire
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    try:
        if not os.path.exists(DATA_FILE):
            print(f"Fichier {DATA_FILE} non trouvé!")
            return
        
        print(f"GPU disponible: {torch.cuda.is_available()}")
        
        # Pipeline
        train_prompts, val_prompts = load_and_prepare_data()
        model, tokenizer = setup_model()
        train_dataset, val_dataset = create_datasets(train_prompts, val_prompts, tokenizer)
        train_model(model, tokenizer, train_dataset, val_dataset)
        
        print("FINE-TUNING TERMINÉ!")
        print(f"Modèle sauvegardé: {OUTPUT_DIR}")
        
    except Exception as e:
        print(f"Erreur: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

FINE-TUNING PHI-3
GPU disponible: True
Chargement des données...
281 exemples chargés
Train: 224, Validation: 57
Chargement du modèle Phi-3...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

Tokenizing train:   0%|          | 0/224 [00:00<?, ? examples/s]

Tokenizing validation:   0%|          | 0/57 [00:00<?, ? examples/s]

Datasets tokenizés
Début de l'entraînement...
Paramètres entraînables: 128


Step,Training Loss
5,2.371700
10,1.940000
15,1.342000
20,0.715000
25,0.545700
30,0.445300
35,0.425800
40,0.384900
45,0.379000
50,0.364800


Modèle sauvegardé dans ./phi3_medical_finetuned
FINE-TUNING TERMINÉ!
Modèle sauvegardé: ./phi3_medical_finetuned


# Utilisation de model Apres le finetuning

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import json
import re

class FinetunedModelTester:
    def __init__(self, model_path):
        self.model_path = model_path
        self.model = None
        self.tokenizer = None
        
    def load_model(self):
        try:
            print("Chargement du modèle fine-tuné...")
            
            # Charger la configuration LoRA
            config = PeftConfig.from_pretrained(self.model_path)
            
            # Charger le tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            # Charger le modèle de base
            base_model = AutoModelForCausalLM.from_pretrained(
                config.base_model_name_or_path,
                torch_dtype=torch.float32,
                trust_remote_code=True,
                device_map="auto", 
            )
            
            # Charger le modèle LoRA
            self.model = PeftModel.from_pretrained(base_model, self.model_path)
            self.model.eval()
            
            print("Modèle chargé avec succès!")
            return True
            
        except Exception as e:
            print(f"Erreur de chargement: {e}")
            return False
    
    def generate_response(self, user_input, max_tokens=120):
        
        prompt = f"""<|user|>
Analyse cette demande médicale et réponds en JSON:
{user_input}
<|end|>
<|assistant|>
"""
        
        try:
            inputs = self.tokenizer(prompt, return_tensors="pt", max_length=400, truncation=True)
            
            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}
            
            input_ids = inputs["input_ids"]
            generated_text = ""
            json_started = False
            json_ended = False
            
            # Génération token par token avec détection de fin JSON
            with torch.no_grad():
                for i in range(max_tokens):
                    outputs = self.model(input_ids)
                    logits = outputs.logits
                    
                    next_token_id = torch.argmax(logits[0, -1, :]).unsqueeze(0).unsqueeze(0)
                    
                    if next_token_id.item() == self.tokenizer.eos_token_id:
                        break
                    
                    # Décoder le nouveau token
                    new_token = self.tokenizer.decode(next_token_id.item(), skip_special_tokens=True)
                    generated_text += new_token
                    
                    # Détecter le début du JSON
                    if "{" in new_token:
                        json_started = True
                    
                    # CONDITION D'ARRÊT : Si on trouve } 
                    if json_started and "}" in new_token:
                        json_ended = True
                        # Ajouter ce token et arrêter
                        input_ids = torch.cat([input_ids, next_token_id], dim=-1)
                        break
                    
                    input_ids = torch.cat([input_ids, next_token_id], dim=-1)
                    
                    if input_ids.shape[1] > 1000:
                        break
            
            # Décoder la réponse finale
            full_response = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
            
            if "<|assistant|>" in full_response:
                response = full_response.split("<|assistant|>")[-1].strip()
                if "<|end|>" in response:
                    response = response.split("<|end|>")[0].strip()
                return response
            else:
                return full_response.replace(prompt, "").strip()
                
        except Exception as e:
            return f"Erreur de génération: {str(e)}"
    
    def parse_json(self, response):
        """parser le JSON"""
        try:
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                return json.loads(json_str)
            else:
                return None
        except:
            return None
    
    def test_examples(self):
        """des exemples prédéfinis"""
        examples = [
            "Je veux un RDV avec Dr Martin demain à 14h",
            "Message urgent pour le cardiologue Dr Mansouri",
            "Je veux réserver une table au restaurant",
            "Consultation avec Dr Sophie lundi matin",
            "Laisser un message au dentiste Dr Khalil"
        ]
        
        print("=== TEST DU MODÈLE FINE-TUNÉ ===\n")
        
        for i, example in enumerate(examples, 1):
            print(f"TEST {i}: {example}")
            print("-" * 50)
            
            # Générer la réponse
            response = self.generate_response(example)
            print(f"RÉPONSE BRUTE:")
            print(response)
            print()
            
            # Essayer de parser le JSON
            parsed = self.parse_json(response)
            if parsed:
                print("JSON PARSÉ:")
                for key, value in parsed.items():
                    print(f"  {key}: {value}")
            else:
                print("PAS DE JSON VALIDE DÉTECTÉ")
            
            print("=" * 60)
            print()

def main():
    model_path = "/kaggle/input/phi3-finetuned-model/phi3_medical_finetuned"
    tester = FinetunedModelTester(model_path)
    
    # Charger le modèle
    if not tester.load_model():
        print("Impossible de charger le modèle. Arrêt du script.")
        return
    
    # Option 1: Tester avec des exemples prédéfinis
    print("Voulez-vous tester avec des exemples prédéfinis ? (y/n)")
    choice = input().lower()
    
    if choice == 'y':
        tester.test_examples()
    
    # Option 2: Tests interactifs
    print("\n=== MODE INTERACTIF ===")
    print("Tapez vos demandes (tapez 'quit' pour arrêter):\n")
    
    while True:
        user_input = input("Votre demande: ")
        
        if user_input.lower() == 'quit':
            break
        
        if user_input.strip():
            print("\nAnalyse en cours...")
            response = tester.generate_response(user_input)
            
            print(f"\nRÉPONSE DU MODÈLE:")
            print(response)
            
            # Parser le JSON
            parsed = tester.parse_json(response)
            if parsed:
                print(f"\nRÉSULTAT STRUCTURÉ:")
                classification = parsed.get('classification', 'INCONNU')
                print(f"Classification: {classification}")
                
                if classification == "RESERVATION":
                    print(f"Heure: {parsed.get('heure', 'NON_SPECIFIE')}")
                    print(f"Date: {parsed.get('date', 'NON_SPECIFIE')}")
                    print(f"Médecin: {parsed.get('nom_docteur', 'NON_SPECIFIE')}")
                elif classification == "MESSAGE":
                    print(f"Message pour: {parsed.get('nom_docteur', 'NON_SPECIFIE')}")
                elif classification == "HORS_SUJET":
                    print("Demande hors domaine médical")
            else:
                print("\nPAS DE JSON VALIDE - Le modèle n'a pas appris le format correct")
            
            print("-" * 50)
        else:
            print("Veuillez saisir une demande valide.")

if __name__ == "__main__":
    main()

Chargement du modèle fine-tuné...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Modèle chargé avec succès!
Voulez-vous tester avec des exemples prédéfinis ? (y/n)


 y


=== TEST DU MODÈLE FINE-TUNÉ ===

TEST 1: Je veux un RDV avec Dr Martin demain à 14h
--------------------------------------------------
RÉPONSE BRUTE:
Analyse cette demande médicale et réponds en JSON:
Je veux un RDV avec Dr Martin demain à 14h
 {"classification": "RESERVATION", "heure": "14h", "date": "demain", "nom_docteur": "Dr Martin"}

JSON PARSÉ:
  classification: RESERVATION
  heure: 14h
  date: demain
  nom_docteur: Dr Martin

TEST 2: Message urgent pour le cardiologue Dr Mansouri
--------------------------------------------------
RÉPONSE BRUTE:
Analyse cette demande médicale et réponds en JSON:
Message urgent pour le cardiologue Dr Mansouri
 {"classification": "MESSAGE", "heure": "NON_SPECIFIE", "date": "NON_SPECIFIE", "nom_docteur": "Dr Mansouri"}

JSON PARSÉ:
  classification: MESSAGE
  heure: NON_SPECIFIE
  date: NON_SPECIFIE
  nom_docteur: Dr Mansouri

TEST 3: Je veux réserver une table au restaurant
--------------------------------------------------
RÉPONSE BRUTE:
Analyse

Votre demande:  je veux faire une reservation de 5 places de table



Analyse en cours...

RÉPONSE DU MODÈLE:
Analyse cette demande médicale et réponds en JSON:
je veux faire une reservation de 5 places de table
 {"classification": "HORS_SUJET", "heure": "NON_SPECIFIE", "date": "NON_SPECIFIE", "nom_docteur": "NON_SPECIFIE"}

RÉSULTAT STRUCTURÉ:
Classification: HORS_SUJET
Demande hors domaine médical
--------------------------------------------------


Votre demande:  quit
